In [ ]:
import os
import fsspec
import pandas as pd
import math

In [ ]:
def convert_size(size_bytes):
   if size_bytes == 0:
       return "0B"
   size_name = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")
   i = int(math.floor(math.log(size_bytes, 1024)))
   p = math.pow(1024, i)
   s = round(size_bytes / p, 2)
   return "%s %s" % (s, size_name[i])

In [ ]:
fname='hytest-internal/nhm_prms_v1_1_gridmet/nhm_prms_v1_1_gridmet.csv'

In [ ]:
fs_write = fsspec.filesystem(
    's3',
    profile='osn-hytest-internal',  ## aws profile name for bucket you are writing data to
    client_kwargs={'endpoint_url': 'https://usgs.osn.mghpcc.org'}
)

In [ ]:
fs_write.ls('hytest-internal/nhm_prms_v1_1_gridmet/')

In [ ]:
total_size = 0
for f in fs_write.ls('hytest-internal/nhm_prms_v1_1_gridmet/raw_data'):
    print(os.path.split(f)[1])
    file_size = fs_write.size(f)
    print(convert_size(file_size))
    total_size+=file_size
print(f'total size of raw data: {convert_size(total_size)}')

In [ ]:
print(f'size of long csv: {convert_size(fs_write.size(fname))}')

In [ ]:
with fs_write.open(fname, mode='rt') as f:
    df = pd.read_csv(f, dtype={'huc12': 'str'}, parse_dates=['date'])
    #print(f.readlines()[1])

In [ ]:
df = df.rename(columns={'date': 'time'})

In [ ]:
df.head(2)

In [ ]:
df.dtypes

In [ ]:
ds = df.set_index(['time', 'huc12']).to_xarray()

In [ ]:
ds

In [ ]:
# write to zarr
zarr_path = 'hytest-internal/nhm_prms_v1_1_gridmet/nhm_prms_v1_1_gridmet.zarr'
outfile=fs_write.get_mapper(zarr_path)
ds.to_zarr(outfile, mode='w', consolidated=True)

In [ ]:
print(f'size of zarr: {convert_size(fs_write.disk_usage(zarr_path))}')